In [153]:
import repository
import configuration

from repository import Repository
from configuration import config
repository = Repository(config)
dataset, labels = repository.get_dataset_and_labels()

In [4]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
from sklearn.cross_validation import ShuffleSplit
from sklearn.grid_search import GridSearchCV

# Ensure that there are no NaNs
dataset = dataset.fillna(-85)
# Split the dataset into training (90 \%) and testing (10 \%)
X_train, X_test, y_train, y_test = train_test_split(dataset, labels,
test_size = 0.1 )

cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2,
random_state=0)

# Define the classifier to use
estimator = SVC(kernel='linear')

# Define parameter space.
gammas = np.logspace(-6, -1, 10)

# Use Test dataset and use cross validation to find bet hyper-parameters.
classifier = GridSearchCV(estimator=estimator, cv=cv,
param_grid=dict(gamma=gammas))
classifier.fit(X_train, [repository.locations.keys().index(tuple(l)) for
l in y_train])

# Test final results with the testing dataset
classifier.score(X_test, [repository.locations.keys().index(tuple(l)) for
l in y_test])

0.5321100917431193

In [70]:
#print dataset.loc[1]
tuple1 = ('00:01:8e:fe:16:6a', -52)
tuple2 = ('00:07:40:07:63:bc', -40)
testpredict = [tuple1, tuple2]
#print testpredict

dataset2 = dataset
dataset2 = dataset2[dataset.index == 45]
classifier.predict(dataset2)

array([153])

In [184]:
from repository import Repository
from configuration import config
import math
repository = Repository(config)
dataset, labels = repository.get_dataset_and_labels()


def diff(a, b):
  b = set(b)
  return [aa for aa in a if aa not in b]


deletecolumns = []

dataset = dataset.fillna(-85)

#set the location labels
numerical_labels = np.array([repository.locations.keys().index(tuple(l)) for l in labels])
numerical_labels = map (int, numerical_labels)
dataset['location',] = numerical_labels

#first clean all AP with maxvalue of -85
#tihs will increase the speed of the second operation
for column in dataset:
    datalist = dataset[column].tolist()
    if max(datalist) <= -85:
        deletecolumns.append(column)
        
dataset = dataset.drop(deletecolumns, axis = 1)

# reset deletecolumns just in case
deletecolumns = []

# we have good and bad AP
goodAP = []
badAP = []

#count amount of different locations we have
loc_amount = len(set(numerical_labels))

for location in range(0, loc_amount):
    
    # check only Data for current location
    location_dataset = dataset.loc[lambda dataset: dataset.location == location, : ]
    
    # amount of measurement data for current location
    measure_amount = len(dataset.loc[lambda dataset: dataset.location == location, : ])

    # if a AP appears >70% in the data for a location its a good access point.
    threshhold = math.ceil(measure_amount*0.7)

    for column in location_dataset:
        if column != "location":
            datalist = location_dataset[column].tolist()
            amount = sum(i > -85 for i in datalist)
            
            if amount > threshhold:
                goodAP.append(column)
            else:
                badAP.append(column)

badAP = set(badAP)
goodAP = set(goodAP)
                
deletecolumns = diff (badAP, goodAP)
print len(deletecolumns)
# delete those columns
# datasetnew = dataset.drop(deletecolumns, axis = 1)

1455


In [203]:
datasetnew = dataset.drop(deletecolumns, axis = 1)

print dataset.shape
print datasetnew.shape

(6540, 1840)
(6540, 385)


In [202]:
# Convert to numerical labels
csv = dataset.sort_values('location')
output = csv.loc[: , ['location', '2c:36:f8:60:23:56','2c:36:f8:60:23:55']]
output = output.replace(-85, "-")
output.to_csv("phil-dataset.csv", sep='\t')